In [17]:
from __future__ import print_function
import math
import random
from collections import defaultdict
from simanneal import Annealer
import numpy as np

def distance(a, b):
    """Calculates distance between two latitude-longitude coordinates."""
    return np.sqrt(np.square(a[0]-b[0])+np.square(a[1]-b[1]))


class TravellingSalesmanProblem(Annealer):

    """Test annealer with a travelling salesman problem.
    """

    # pass extra data (the distance matrix) into the constructor
    def __init__(self, state, distance_matrix):
        self.distance_matrix = distance_matrix
        super(TravellingSalesmanProblem, self).__init__(state)  # important!

    def move(self):
        """Swaps two cities in the route."""
        # no efficiency gain, just proof of concept
        # demonstrates returning the delta energy (optional)
        initial_energy = self.energy()

        a = random.randint(0, len(self.state) - 1)
        b = random.randint(0, len(self.state) - 1)
        self.state[a], self.state[b] = self.state[b], self.state[a]

        return self.energy() - initial_energy

    def energy(self):
        """Calculates the length of the route."""
        e = 0
        for i in range(len(self.state)):
            e += self.distance_matrix[self.state[i-1]][self.state[i]]
        return e



# latitude and longitude for the twenty largest U.S. cities
cities=[(0,0),(10,10),(0,10),(10,0)]
# initial state, a randomly-ordered itinerary
init_state = list(cities)
random.shuffle(init_state)
distance_matrix = defaultdict(dict)
for ka in cities:
    for kb in cities:
        distance_matrix[ka][kb] = 0.0 if kb == ka else distance(ka, kb)

tsp = TravellingSalesmanProblem(init_state, distance_matrix)
tsp.set_schedule(tsp.auto(minutes=1))
# since our state is just a list, slice is the fastest way to copy
tsp.copy_strategy = "slice"
state, e = tsp.anneal()
print()
print("%i mile route:" % e)
print(state)

def read_sudoku_from_file(file_name):
    """returns sudoku as np.array"""
    with open(file_name,"r") as file:
        lines=file.readlines()
        sudoku=np.zeros((9,9))
        fixed={}
        for i,line in enumerate(lines):
            x=[-1  if line[i]=='x' else int(line[i]) if line[i]!='\n' else 0 for i in range(len(line)-1)]
            for j in range(len(x)):
                if x[j]!=-1:
                    fixed[(i,j)]=x[j]
                sudoku[i,j]=x[j]    
        return sudoku,fixed
def next_sudoku(old_state,options):
#     Choose randomly i and j, such that (a) 1 ≤ i, j ≤ n^2 , and (b) cell i, j is non-fixed.
# Choose randomly k and l, such that (a) cell k, l is in the same square as cell i, j , (b) cell k, l is non-fixed,
# and (c) cell k, l ≠ cell i, j .
# Swap cell i, j with cell k, l .
    state=old_state.copy()
    i,j=np.random.randint(0,state.shape[0],size=2)
    k,l=np.random.randint(0,state.shape[0],size=2)
    state[i][j],state[k][l]=state[k][l],state[i][j]
    return state
def same_val_count(arr):
    res=0
#     rows
    for i in range(arr.shape[0]):
        counts=np.unique(arr[i],return_counts=True)[1]
        res+=np.sum(np.where(counts>1,counts,0))
    for i in range(arr.shape[1]):
        counts=np.unique(arr[:,i],return_counts=True)[1]
        res+=np.sum(np.where(counts>1,counts,0))
    return res
def sudoku_cost(sudoku):
    res=same_val_count(sudoku)
    for i in range(0,9,3):
        for k in range(0,9,3):
            res+=same_val_count(sudoku[i:i+3,k:k+3])
    return res
sudoku,fixed=read_sudoku_from_file('sample_sudoku')
print(fixed)
# fill with random values, such that each value beetwen 1 and n^2 occurs only once 
# in each 3x3 square
# TODO
# end sudoku and tsp
def fix(array):
    c=1
    occupied=[0 for i in range(10)]
    occupied[0]=1
    for i in range(array.shape[0]):
        for j in range(array.shape[1]):
            if array[i][j]!=-1:
                occupied[int(array[i][j])]=1
    for i in range(array.shape[0]):
        for j in range(array.shape[1]):
            if array[i][j]==-1:
                for k in range(len(occupied)):
                    if occupied[k]==0:
                        array[i][j]=k
                        occupied[k]=1
                        break
    return array
for i in range(0,9,3):
    for k in range(0,9,3):
        sudoku[i:i+3,k:k+3]=fix(sudoku[i:i+3,k:k+3].copy())
print(sudoku)
print(sudoku_cost(sudoku))
x=anneal(max_steps=10000000,t_init=720000000,t_end=2.3,state_init=sudoku,save_animation=True,
         neighbour=next_sudoku,cost = sudoku_cost,cooling_type='expotential',swap_type='consecutive')
print(x[0],x[1])

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.67000         40.00    50.70%     0.00%     0:00:01     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.67000         40.00    50.14%     0.00%     0:00:21     0:00:00


40 mile route:
[(10, 10), (0, 10), (0, 0), (10, 0)]
